In [1]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import BeitFeatureExtractor, BeitForImageClassification
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn import model_selection, metrics
import cv2
from PIL import Image
import random

In [2]:
transform_train = transforms.Compose([
        
         transforms.Resize((224, 224)),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.5, 0.5, 0.5], 
                              std=[0.5, 0.5, 0.5]),
])

In [3]:
data_path = '../input/neurowood/trainset'
train_data = datasets.ImageFolder(data_path, transform=transform_train)

In [4]:
def seed_everything(seed = 1234):
     random.seed(seed)
     
     os.environ['PYTHONHASHSEED'] = str(seed)
     
     np.random.seed(seed)     
     
     torch.manual_seed(seed)
     
     torch.cuda.manual_seed(seed)
     
     torch.backends.cudnn.deterministic = True



In [5]:
seed_everything()

In [6]:
def subset_ind(dataset, ratio: float):
    return np.random.choice(len(dataset), size=int(ratio*len(dataset)), replace=False)

In [7]:
val_size = 0.01
val_inds = subset_ind(train_data, val_size)
train_dataset = Subset(train_data, [i for i in range(len(train_data)) if i not in val_inds])
val_dataset = Subset(train_data, val_inds)
batch_size = 40
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=0)

print(f'training size: {len(train_dataset)}\nvalidation size: {len(val_dataset)}')

In [ ]:
# batch_size = 8
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=0)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=0)

In [8]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()  # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs).logits

                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return model

In [ ]:
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', output_hidden_states=True)
#model = BeitForImageClassification.from_pretrained("microsoft/beit-base-patch16-224", output_hidden_states=True)
for param in model.parameters():
    param.requires_grad = False
model.classifier = torch.nn.Linear(model.classifier.in_features, 3)
device = 'cuda:0'
model.to(device)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
train_model(model, loss, optimizer, scheduler, num_epochs=7)

In [20]:
test = os.listdir('../input/neurowood/testset')

In [21]:
class MyDataset(Dataset):
    def __init__(self, image_paths, transform):
        self.image_paths = image_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        
        image_filepath = self.image_paths[idx]
        
        image = cv2.imread('../input/neurowood/testset/'+image_filepath)
    
        image = Image.fromarray(image)
        image = self.transform(image)
       
        return image, image_filepath

In [22]:
test_dataset = MyDataset(test, transform_train)

In [23]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [24]:
names = []
predicts = []
with torch.no_grad():
  
  for inputs, name in tqdm(test_dataloader):
                
                inputs = inputs.to(device)
                preds = model(inputs).logits
                preds_class = preds.argmax(dim=1)
                names.extend(name)
                predicts.extend(preds_class.tolist())

In [27]:
import pandas as pd
df = pd.DataFrame([])
for i, k in zip(names, predicts):
    
  df = df.append(pd.concat([pd.Series(i), pd.Series(k)], axis=1))

 

In [28]:
df.columns = ['id', 'class']
df['class'] = df['class'].replace({1:3, 0:1, 2:0})
df['id'] = df['id'].str.replace('.png','')
df['id'] = df['id'].str.replace('.JPG','')
df['id'] = df['id'].str.replace('.jpg','')
df['id'] = df['id'].apply(lambda x: int(x))
df.sort_values('id', inplace=True, ascending=True)
df.to_csv('sub(17.04.2022)(12).csv', index=False, sep=',')

In [56]:
#Feature extracting for classifier, optional
#embeds for training
names1 = []
embs1 = []
with torch.no_grad():
  
  for inputs, name in tqdm(train_dataloader):
                
                inputs = inputs.to(device)
                emb = model(inputs)
                outputs = emb.hidden_states[12][:,0,:].detach().cpu().numpy()
                names1.extend(name)
                embs1.extend(outputs)
embedstrain=pd.DataFrame([i.tolist() for i in embs1])
m = pd.DataFrame([i.tolist() for i in names1])
embedstrain['lab'] = m 
embedstrain.to_csv('embedstrain.csv')

In [57]:
#embeds for testing
names2 = []
embs2 = []
with torch.no_grad():
  
  for inputs, name in tqdm(test_dataloader):
                
                inputs = inputs.to(device)
                emb = model(inputs)
                outputs = emb.hidden_states[12][:,0,:].detach().cpu().numpy()
                
                
                names2.extend(name)
                embs2.extend(outputs)
embedstest = pd.DataFrame([i.tolist() for i in embs2])
m = pd.Series(names2)
embedstest['id'] = m
embedstest.to_csv('embedstest.csv')